#Homework

    Test Whether the Parameters are in the Model
    
    Add code to add the intercept to the model without it being in the dataset

### Create the Dataset

In [1]:
using DataFrames
inter = fill(1,10)
A = [1,1,1,1,1,2,2,2,2,2]
B = [1,1,2,2,2,1,1,1,2,2]
C = [1,2,1,2,2,1,2,2,1,2]
df1 = DataFrame(intercept=inter,A=A,B=B,X=randn(10),W=randn(10),y=randn(10))
df2 = DataFrame(A=A,B=B,X=randn(10),W=randn(10),y=randn(10))

,A,B,X,W,y
1,1,1,-0.46140639263679695,-1.0642358643418959,0.5009107796769866
2,1,1,0.29892411132040064,-0.02025599600345363,0.3749778510702613
3,1,2,2.388163806001887,0.5359049550240381,0.9627972572543042
4,1,2,1.5574349516933037,0.5452760547313114,1.0760945590997033
5,1,2,-1.693604570912408,0.6400866106883827,-0.26493929091749846
6,2,1,0.23775869320175416,-0.9706975035086898,0.5999386867574924
7,2,1,-0.6208002848122265,-0.21542868551511352,-0.3859737581156107
8,2,1,-1.215302743911465,-0.37115347399230375,0.1176585550481973
9,2,2,-0.7219159519736128,-1.725868432024615,-0.46805209746238285
10,2,2,-1.08399973884285,-0.4352134549173871,1.323968872695453


In [2]:
names(df1)

6-element Array{Symbol,1}:
 :intercept
 :A        
 :B        
 :X        
 :W        
 :y        

###Main Code

####Define the MME and ModelTerm Objects

In [3]:
type ModelTerm 
    trmStr::String
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{String,1}
    val::Array{Float64,1}
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type MME
    modelEquation::String
    modelTerms::Array{ModelTerm,1}
    lhs::Symbol
    covVec::Array{Symbol,1}
    mmeLhs
    mmeRhs
    X
end;

####The Gauss-Seidel, Jacobi and Dictionary Functions

In [4]:
function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    iter = 0
    diff = 1.0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end;

####Create the ModelTerms

In [5]:
function getTerm(trmStr)
    trm = ModelTerm(trmStr,0,[],[],[],spzeros(0,0),[])
    if length(trmStr)==1
        trm.nFactors = 1
        trm.factors  = [symbol(strip(trmStr))]
    else
        factorVec = split(trmStr,"*")
        trm.nFactors = length(factorVec)
        trm.factors = [symbol(strip(f)) for f in factorVec]
    end
    return trm
end;

####Create the MME based on the Equation (but not yet adding the data)

In [6]:
function initMME(modelEquation::String)
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if modelEquation==""
        println("modelEquation is empty\n")
        return
    end
    lhsRhs = split(modelEquation,"=")
    lhs = symbol(strip(lhsRhs[1]))
    rhs = strip(lhsRhs[2])
    rhsVec = split(rhs,"+")    
    modelTerms = [getTerm(strip(trmStr)) for trmStr in rhsVec]
    return MME(modelEquation,modelTerms,lhs,[],0,0,0)
end;

####Add the Data to the ModelTerms Object

In [7]:
function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(String,nObs)
    trm.val = Array(Float64,nObs)
    myDf = df[trm.factors]
    if trm.factors[1] in mme.covVec
        str = fill(string(trm.factors[1]),nObs)
        val = df[trm.factors[1]]
    else
        str = [string(i) for i in df[trm.factors[1]]]
        val = fill(1.0,nObs)
    end
    for i=2:trm.nFactors
        if trm.factors[i] in mme.covVec
            str = str .* fill("x"*string(trm.factors[i]),nObs)
            val = val .* df[trm.factors[i]]
        else
            str = str .* fill("x",nObs) .* [string(j) for j in df[trm.factors[i]]]
            val = val .* fill(1.0,nObs)
        end
    end
    trm.str = str
    trm.val = val
end;

####Create the X matrix based on the ModelTerms

In [8]:
function getX(trm)
    dict,trm.names  = mkDict(trm.str)
    xj    = int([dict[i] for i in trm.str])
    xi    = 1:size(trm.str,1)
    trm.X = sparse(xi,xj,trm.val)  
end;

####Create the Left Hand Side and Right Hand Side of  the MME

In [9]:
function getMME(mme::MME, df::DataFrame)
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y      = df[mme.lhs]
    mme.mmeLhs = X'X
    mme.mmeRhs = X'y
    mme.X = X
end;

####Get the names of each of the ModelTerms (to print out solutions later)

In [10]:
function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end; 

####Create function to specify which parameters are covariates, the default is a class variable

In [11]:
function covList(mme::MME, covStr::String)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end;

##Test Whether the Model Parameters are in the DataFrame (except intercept)

    Get the variable on the left hand side (i.e. y)
    
    Add to that array the modelTerm factors and get the unique ones
    
    Set boo to true
    
    If the parameter is not the intercept, try to get each of the unique parameters from the DataFrame
        If the parameter is in the DataFrame, do nothing
        If the parameter is not in the DataFrame, print out the error message and set boo to false
        
    Return boo

In [12]:
function testTerms(mme::MME, df::DataFrame)
    a = mme.lhs
    for i in 1:length(mme.modelTerms)
        a = [a,mme.modelTerms[i].factors]
    end
    b = unique(a)
    boo = true
    for s in b
        if(s != :intercept)
           try
               df[s]
           catch
               println("Parameter ",s," is not in the given DataFrame")
               boo = false
           end
        end
    end
    return boo
end;

##Change getData to Include an Intercept

    Need to test whether the factor is "intercept", then fill str and val with "intercept" and 1.0, respectively.
    
    For all other ModelTerms, fill as normal.

In [13]:
function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(String,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)
            str = fill(string(trm.factors[1]),nObs)
            val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("x"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("x",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end;

###Change the getSol Codes to Call testTerms function

    First test the terms in the model
        If testTerms returns true (i.e. the parameters, minus intercept, are all in the DataFrame), get the solution.
        If testTerms returns false, do nothing (but testTerms will print out an error message).

In [14]:
function getSolJ(mme::MME, df::DataFrame)
    if testTerms(mme,df)
       if size(mme.mmeRhs)==() 
           getMME(mme,df)
       end
       p = size(mme.mmeRhs,1)
       return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
    end
end

function getSolG(mme::MME, df::DataFrame)
    if testTerms(mme,df)
       if size(mme.mmeRhs)==() 
           getMME(mme,df)
       end
       p = size(mme.mmeRhs,1)
       return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
    end
end;

##Testing!

####All parameters are in the model:

In [15]:
mme = initMME("y = A + B + A*B")
getSolG(mme,df1)

1 1.0322984501915584e-31


8x2 Array{Any,2}:
 "A: 1"       0.439629 
 "A: 2"       0.135026 
 "B: 1"      -0.0859639
 "B: 2"       0.0859639
 "A*B: 1x1"   0.416476 
 "A*B: 1x2"  -0.306306 
 "A*B: 2x1"  -0.277651 
 "A*B: 2x2"   0.459458 

####Including parameters that are not in the model:

In [16]:
mme2 = initMME("y = A + B + A*D*E")
getSolG(mme2,df1)

Parameter D is not in the given DataFrame
Parameter E is not in the given DataFrame


####Including a covariate:

In [17]:
mme3 = initMME("y = W")
covList(mme3,"W")
getSolG(mme3,df1)

1 0.0


1x2 Array{Any,2}:
 "W: W"  -0.0388144

####Including a y variable that is not in the DataFrame:

In [18]:
mme4 = initMME("P = X + L")
getSolG(mme4,df1)

Parameter P is not in the given DataFrame
Parameter L is not in the given DataFrame


####Including the intercept when it is in the DataFrame:

In [19]:
mme5 = initMME("y = intercept + X + W")
covList(mme5,"X W")
getSolG(mme5,df1)

1 0.03548362152477033
2 0.0011581671566600925
3 0.00022346991808391706
4 4.7200284586298634e-5
5 6.965148422269336e-6
6 8.040727332262058e-7


3x2 Array{Any,2}:
 "intercept: intercept"  0.304917   
 "X: X"                  0.000784314
 "W: W"                  0.0541937  

####Including the intercept when it is not in the DataFrame:

In [20]:
mme6 = initMME("y = intercept + W")
covList(mme6,"X W")
getSolG(mme6,df2)

1 0.18386879798089514
2 0.004020744046679046
3 8.792346970465719e-5
4 1.9226631775504723e-6
5 4.204376495519241e-8


2x2 Array{Any,2}:
 "intercept: intercept"  0.454869
 "W: W"                  0.23092 

####Including the intercept and a parameter not in the DataFrame:

In [21]:
mme7 = initMME("y = intercept + X + W + J")
covList(mme7,"X W")
getSolG(mme7,df2)

Parameter J is not in the given DataFrame
